### 2021_09_29_5

### 코로나 데이터와 백신 데이터 합병

In [2]:
import pandas as pd

### 데이터 불러오기
* 코로나 : ./CSV/2021-09-28_corona.csv
* 백신 : ./CSV/20210929_14_vaccine_bloomberg.csv
* 나라 코드 : ./CSV/country.csv

In [3]:
# excel 데이터는 pd.read_excel로 가져올 수 있음.

corona = pd.read_csv("./CSV/2021-09-28_corona.csv")
vaccine = pd.read_csv("./CSV/20210929_14_vaccine_bloomberg.csv")
country_code = pd.read_csv("./CSV/country.csv", encoding = "euc-kr")

In [4]:
print("corona shape :", corona.shape)
print("vaccine shape :", vaccine.shape)
print("country_code shape :", country_code.shape)

corona shape : (222, 13)
vaccine shape : (207, 6)
country_code shape : (227, 4)


In [5]:
country_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   국문(코로나보드)  221 non-null    object
 1   영문(블룸버그)   202 non-null    object
 2   나라         12 non-null     object
 3   비고_뜻       30 non-null     object
dtypes: object(4)
memory usage: 7.2+ KB


In [6]:
corona.head(3)

,국가,치명(%),완치(%),발생률,인구수,확진자_합계,확진자_1일,사망자_합계,사망자_1일,완치_합계,완치_1일,위중증_합계,위중증_1일
0,미국🇺🇸,1.6,76.0,133058,331002651,43942335,187382.0,709119,2781.0,33394833.0,208564.0,21819.0,-275.0
1,인도🇮🇳,1.3,97.8,24431,1380004385,33714241,21093.0,447699,337.0,32976760.0,28144.0,8944.0,NaN
2,브라질🇧🇷,2.8,95.3,100592,212559417,21366395,14423.0,594702,218.0,20361191.0,20818.0,8318.0,NaN


In [7]:
vaccine.head(3)

,국가,백신접종수,Enough_percent_people,1차접종,2차접종,일별접종수
0,Global Total,6.199444e+09,-999.0,-999.0,-999.0,29152868.0
1,Mainland China,2.203185e+09,78.7,78.6,73.0,3171286.0
2,India,8.765386e+08,32.0,46.7,16.8,7178901.0


In [8]:
country_code.head(3)

,국문(코로나보드),영문(블룸버그),나라,비고_뜻
0,가나,Ghana,NaN,NaN
1,가봉,Gabon,NaN,NaN
2,가이아나,Guyana,NaN,NaN


### 코로나 데이터 국가명에서 한글 추출

In [9]:
corona["국가"].str.extract('([ᄀ-ᅵ가-힣]+)').head(3)

,0
0,미국
1,인도
2,브라질


In [10]:
corona["한글표기"] = corona["국가"].str.extract('([ᄀ-ᅵ가-힣]+)', expand = False)
corona.columns

Index(['국가', '치명(%)', '완치(%)', '발생률', '인구수', '확진자_합계', '확진자_1일', '사망자_합계',
       '사망자_1일', '완치_합계', '완치_1일', '위중증_합계', '위중증_1일', '한글표기'],
      dtype='object')

### 새로운 데이터 프레임 생성
* 컬럼 순서 변경

In [11]:
col = ['국가', '한글표기', '위중증_합계', '위중증_1일', '치명(%)', '완치(%)',
       '발생률', '인구수', '확진자_합계', '확진자_1일', '사망자_합계','사망자_1일', 
       '완치_합계', '완치_1일']

new_corona = corona[col].copy()
new_corona.head(3)

,국가,한글표기,위중증_합계,위중증_1일,치명(%),완치(%),발생률,인구수,확진자_합계,확진자_1일,사망자_합계,사망자_1일,완치_합계,완치_1일
0,미국🇺🇸,미국,21819.0,-275.0,1.6,76.0,133058,331002651,43942335,187382.0,709119,2781.0,33394833.0,208564.0
1,인도🇮🇳,인도,8944.0,NaN,1.3,97.8,24431,1380004385,33714241,21093.0,447699,337.0,32976760.0,28144.0
2,브라질🇧🇷,브라질,8318.0,NaN,2.8,95.3,100592,212559417,21366395,14423.0,594702,218.0,20361191.0,20818.0


### country_code 컬럼명 변경

In [12]:
country_code.columns

Index(['국문(코로나보드)', '영문(블룸버그)', '나라', '비고_뜻'], dtype='object')

In [13]:
country_code.columns = ["han_code", "eng_code", "country", "etc"]
country_code.head(3)

,han_code,eng_code,country,etc
0,가나,Ghana,NaN,NaN
1,가봉,Gabon,NaN,NaN
2,가이아나,Guyana,NaN,NaN


### 두 데이터 연결(merge)

In [14]:
df_corona = new_corona.merge(country_code, left_on = "한글표기",
                            right_on = "han_code")
df_corona.head(3)

,국가,한글표기,위중증_합계,위중증_1일,치명(%),완치(%),발생률,인구수,확진자_합계,확진자_1일,사망자_합계,사망자_1일,완치_합계,완치_1일,han_code,eng_code,country,etc
0,미국🇺🇸,미국,21819.0,-275.0,1.6,76.0,133058,331002651,43942335,187382.0,709119,2781.0,33394833.0,208564.0,미국,U.S.,NaN,NaN
1,인도🇮🇳,인도,8944.0,NaN,1.3,97.8,24431,1380004385,33714241,21093.0,447699,337.0,32976760.0,28144.0,인도,India,NaN,NaN
2,브라질🇧🇷,브라질,8318.0,NaN,2.8,95.3,100592,212559417,21366395,14423.0,594702,218.0,20361191.0,20818.0,브라질,Brazil,NaN,NaN


### 필요 없는 컬럼 삭제
* 국가, han_code, country, etc

In [15]:
df_corona.drop(["국가", "han_code", "country", "etc"], axis = 1, inplace = True)

In [16]:
df_corona.head(3)

,한글표기,위중증_합계,위중증_1일,치명(%),완치(%),발생률,인구수,확진자_합계,확진자_1일,사망자_합계,사망자_1일,완치_합계,완치_1일,eng_code
0,미국,21819.0,-275.0,1.6,76.0,133058,331002651,43942335,187382.0,709119,2781.0,33394833.0,208564.0,U.S.
1,인도,8944.0,NaN,1.3,97.8,24431,1380004385,33714241,21093.0,447699,337.0,32976760.0,28144.0,India
2,브라질,8318.0,NaN,2.8,95.3,100592,212559417,21366395,14423.0,594702,218.0,20361191.0,20818.0,Brazil


In [17]:
vaccine.head(3)

,국가,백신접종수,Enough_percent_people,1차접종,2차접종,일별접종수
0,Global Total,6.199444e+09,-999.0,-999.0,-999.0,29152868.0
1,Mainland China,2.203185e+09,78.7,78.6,73.0,3171286.0
2,India,8.765386e+08,32.0,46.7,16.8,7178901.0


### df_corona, vaccine merge
* df_corona_all 생성

In [18]:
df_corona_all = df_corona.merge(vaccine, left_on = "eng_code",
                               right_on = "국가")

In [19]:
df_corona_all.head(3)

,한글표기,위중증_합계,위중증_1일,치명(%),완치(%),발생률,인구수,확진자_합계,확진자_1일,사망자_합계,사망자_1일,완치_합계,완치_1일,eng_code,국가,백신접종수,Enough_percent_people,1차접종,2차접종,일별접종수
0,미국,21819.0,-275.0,1.6,76.0,133058,331002651,43942335,187382.0,709119,2781.0,33394833.0,208564.0,U.S.,U.S.,391152580.0,61.1,64.4,55.8,701422.0
1,인도,8944.0,NaN,1.3,97.8,24431,1380004385,33714241,21093.0,447699,337.0,32976760.0,28144.0,India,India,876538600.0,32.0,46.7,16.8,7178901.0
2,브라질,8318.0,NaN,2.8,95.3,100592,212559417,21366395,14423.0,594702,218.0,20361191.0,20818.0,Brazil,Brazil,233261970.0,56.5,71.3,41.9,1317163.0


### 국가 컬럼 삭제

In [20]:
df_corona_all.drop(["국가"], axis = 1, inplace = True)

In [21]:
df_corona_all.head(3)

,한글표기,위중증_합계,위중증_1일,치명(%),완치(%),발생률,인구수,확진자_합계,확진자_1일,사망자_합계,사망자_1일,완치_합계,완치_1일,eng_code,백신접종수,Enough_percent_people,1차접종,2차접종,일별접종수
0,미국,21819.0,-275.0,1.6,76.0,133058,331002651,43942335,187382.0,709119,2781.0,33394833.0,208564.0,U.S.,391152580.0,61.1,64.4,55.8,701422.0
1,인도,8944.0,NaN,1.3,97.8,24431,1380004385,33714241,21093.0,447699,337.0,32976760.0,28144.0,India,876538600.0,32.0,46.7,16.8,7178901.0
2,브라질,8318.0,NaN,2.8,95.3,100592,212559417,21366395,14423.0,594702,218.0,20361191.0,20818.0,Brazil,233261970.0,56.5,71.3,41.9,1317163.0


### 최종 데이터 프레임 생성
* 컬럼 순서 지정
    * ['한글표기', 'eng_code', '위중증', '치명(%)', '완치(%)', '발생률', '인구수', '확진자_합계', '확진자_1일,'사망자합계', '사망자1일', '완치합계', '완치1일', '백신접종수', 'Enough_for_percent_of_people:', '1차접종', '2차접종', '일별접종수' ]


In [22]:
df_corona_all.columns

Index(['한글표기', '위중증_합계', '위중증_1일', '치명(%)', '완치(%)', '발생률', '인구수', '확진자_합계',
       '확진자_1일', '사망자_합계', '사망자_1일', '완치_합계', '완치_1일', 'eng_code', '백신접종수',
       'Enough_percent_people', '1차접종', '2차접종', '일별접종수'],
      dtype='object')

In [23]:
sel = ['한글표기', 'eng_code', '위중증_합계', '위중증_1일', '치명(%)', '완치(%)', 
       '발생률', '인구수', '확진자_합계', '확진자_1일', '사망자_합계', '사망자_1일', 
       '완치_합계', '완치_1일', '백신접종수', 'Enough_percent_people', 
       '1차접종', '2차접종', '일별접종수']
result_df = df_corona_all[sel].copy()
result_df.head(3)

,한글표기,eng_code,위중증_합계,위중증_1일,치명(%),완치(%),발생률,인구수,확진자_합계,확진자_1일,사망자_합계,사망자_1일,완치_합계,완치_1일,백신접종수,Enough_percent_people,1차접종,2차접종,일별접종수
0,미국,U.S.,21819.0,-275.0,1.6,76.0,133058,331002651,43942335,187382.0,709119,2781.0,33394833.0,208564.0,391152580.0,61.1,64.4,55.8,701422.0
1,인도,India,8944.0,NaN,1.3,97.8,24431,1380004385,33714241,21093.0,447699,337.0,32976760.0,28144.0,876538600.0,32.0,46.7,16.8,7178901.0
2,브라질,Brazil,8318.0,NaN,2.8,95.3,100592,212559417,21366395,14423.0,594702,218.0,20361191.0,20818.0,233261970.0,56.5,71.3,41.9,1317163.0


### 최종 데이터 프레임 컬럼명 변경

In [24]:
result_df.columns

Index(['한글표기', 'eng_code', '위중증_합계', '위중증_1일', '치명(%)', '완치(%)', '발생률', '인구수',
       '확진자_합계', '확진자_1일', '사망자_합계', '사망자_1일', '완치_합계', '완치_1일', '백신접종수',
       'Enough_percent_people', '1차접종', '2차접종', '일별접종수'],
      dtype='object')

In [25]:
result_df.columns = ['국가명', 'eng_code', '위중증_합계', '위중증_1일', 
                     '치명(%)', '완치(%)', '발생률', '인구수', '확진자_합계', 
                     '확진자_1일', '사망자_합계', '사망자_1일', '완치_합계', 
                     '완치_1일', '백신접종', '접종가능률(인구)', 
                     '1차접종', '2차접종', '일별접종수']

In [26]:
result_df.head(3)

,국가명,eng_code,위중증_합계,위중증_1일,치명(%),완치(%),발생률,인구수,확진자_합계,확진자_1일,사망자_합계,사망자_1일,완치_합계,완치_1일,백신접종,접종가능률(인구),1차접종,2차접종,일별접종수
0,미국,U.S.,21819.0,-275.0,1.6,76.0,133058,331002651,43942335,187382.0,709119,2781.0,33394833.0,208564.0,391152580.0,61.1,64.4,55.8,701422.0
1,인도,India,8944.0,NaN,1.3,97.8,24431,1380004385,33714241,21093.0,447699,337.0,32976760.0,28144.0,876538600.0,32.0,46.7,16.8,7178901.0
2,브라질,Brazil,8318.0,NaN,2.8,95.3,100592,212559417,21366395,14423.0,594702,218.0,20361191.0,20818.0,233261970.0,56.5,71.3,41.9,1317163.0


### corona vaccine merge csv 파일 생성

In [27]:
result_df.to_csv("./CSV/corona_vaccine_merge.csv")